<a href="https://colab.research.google.com/github/Samrudh29/IR-HomeWork2/blob/main/Copy_of_CrossLanguageEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

In class, we discussed how we can reduce the dimensionality of word representations from their original vector space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classification or retrieval models.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [39]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the plays of William Shakespeare.

In [40]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2022-04-18 03:09:08--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt [following]
--2022-04-18 03:09:08--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/shakespeare_plays.txt
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt.3’

shakespeare_plays.t 100%[===================>]   4.53M  3.12MB/s    in 1.5s    

2022-04-18 03:09:11 (3.12 MB/s) - ‘shakespeare_plays.txt.3’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [41]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [19]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.8366804122924805),
 ('york', 0.7633033394813538),
 ('prince', 0.7581432461738586),
 ('duke', 0.7502011060714722),
 ('clarence', 0.7362352609634399),
 ('warwick', 0.7183172702789307),
 ('gloucester', 0.714714765548706),
 ('son', 0.7139991521835327),
 ('duchess', 0.7110257148742676),
 ('princess', 0.7104967832565308)]

In other words, we want a vector close to `king` and `woman` but subtracting the dimensions that are important to `man`, i.e., `queen`. Other words are mostly noble titles important in Shakespeare's "history" plays.

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [42]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9609982967376709),
 ('imogen', 0.9471182823181152),
 ('emilia', 0.9325530529022217),
 ('lucio', 0.926884114742279),
 ('iago', 0.9254346489906311),
 ('troilus', 0.9247232675552368),
 ('cassio', 0.9240168929100037),
 ('pisanio', 0.9216016530990601),
 ('cleopatra', 0.9209161996841431),
 ('helena', 0.9156539440155029)]

If you know the play, you might see some familiar names.

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [43]:
model.wv.similarity('othello', 'desdemona')

0.96099836

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [44]:
## TODO: Implement cosim
#Using norm and dot function of numpy for calculation
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  mul_v1_v2 = np.dot(v1, v2) 
  l2_v1 = np.linalg.norm(v1)
  l2_v2 = np.linalg.norm(v2)
  co_sim = mul_v1_v2/(l2_v1 * l2_v2)
  return co_sim

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.9609984

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [23]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ar.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.ru.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.zh.vec

--2022-04-18 01:54:19--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec [following]
--2022-04-18 01:54:20--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.en.vec
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.1’

30k.en.vec.1        100%[===================>]  64.54M  14.7MB/s    in 5.4s    

2022-04-18 01:54:26 (11.9 MB/s) - ‘30k.en.vec.1’ saved [67681172/67681172]

--2022-04-18 01:54:26--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [24]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
arvec = ftvectors('30k.ar.vec')
devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
#esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [25]:
## TODO: implement this search function

def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  mostSimilar_word = ''
  similarity_score = 0
  
  for word in vecDict.keys():
    sim = cosim(vec,vecDict.get(word))
    if sim > similarity_score:
      similarity_score = sim
      mostSimilar_word = word
  return (mostSimilar_word, similarity_score)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.593718413875964),
 ('matrice', 0.5088361302065517),
 ('physique', 0.4555543434796394),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [26]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
links = [s.split() for s in open('links.tab')]

--2022-04-18 01:55:13--  http://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab [following]
--2022-04-18 01:55:14--  https://www.ccs.neu.edu/home/dasmith/courses/cs6200/links.tab
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab.2’

links.tab.2         100%[===================>]   1.34M  1.48MB/s    in 0.9s    

2022-04-18 01:55:16 (1.48 MB/s) - ‘links.tab.2’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [27]:
links[302]

['academy', 'fr', 'académie']

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [ ]:
## TODO: Compute English-French Wikipedia retrieval accuracy.

accuracy = 0
baselineAccuracy = 0
total_word_count = 0
matching_words = 0
similar_words = 0
for row in links:
    if row[1] == 'fr':
        if row[0] in envec.keys():
            total_word_count += 1
            if row[0] == row[2]:
                matching_words += 1   
            similar, _ = mostSimilar(envec.get(row[0]), frvec)
            if similar == row[2]:
                similar_words += 1

baselineAccuracy = matching_words/total_word_count
accuracy = similar_words/total_word_count
print('Base Line Accuracy:',baselineAccuracy)
print('Accuracy:',accuracy)

Base Line Accuracy: 0.6742324450298915
Accuracy: 0.5359205593271862


**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [ ]:
## TODO: Compute English-German Wikipedia retrieval accuracy.
accuracy = 0
baselineAccuracy = 0
total_word_count = 0
matching_words = 0
similar_words = 0
for row in links:
    if row[1] == 'ar':
        if row[0] in envec.keys():
            total_word_count += 1
            if row[0] == row[2]:
                matching_words += 1   
            similar, _ = mostSimilar(envec.get(row[0]), arvec)
            if similar == row[2]:
                similar_words += 1

baselineAccuracy = matching_words/total_word_count
accuracy = similar_words/total_word_count
print('Base Line Accuracy:',baselineAccuracy)
print('Accuracy:',accuracy)

Base Line Accuracy: 0.006582155046318869
Accuracy: 0.2067284251584593


**TODO**: Find the 10 nearest neighbors of each English term to compute "recall at 10" and "mean reciprocal rank at 10".

In [28]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in French and some other language.

from itertools import islice
import pandas as pd
def mostSimilarTop10(vec, vecDict):
  similarity_dict = {} 
  for word in vecDict.keys():
    x = cosim(vec,vecDict.get(word))
    similarity_dict[word] = x
  similarity_dict = dict(sorted(similarity_dict.items(), key=lambda k: k[1], reverse=True))
  return (list(similarity_dict.keys())[:10])

In [ ]:
df = pd.DataFrame(columns = ['word', 'similar_words_collection','translated_word', 'recall_value','reciprocal_rank'])
for row in links:
  if row[1] == 'fr' :
    if row[0] in envec.keys():
      similar_words = mostSimilarTop10(envec.get(row[0]), frvec)
      df.loc[len(df.index)] = [row[0],similar_words,row[2],999,999] 


In [18]:
recall_values = []
reciprocal_rank = []
for translated_word, words in zip(df['translated_word'], df['similar_words_collection']):

  if translated_word in words:
    recall_values.append(1)
    reciprocal_rank.append(1 / (words.index(translated_word) + 1))
  else:
    recall_values.append(0)
    reciprocal_rank.append(0)

df['recall_value'] = recall_values
df['reciprocal_rank'] = reciprocal_rank
mean_reciprocal_rank_fr = sum(df['reciprocal_rank'])/len(df)
print(mean_reciprocal_rank_fr)

0.5646970954423574


In [19]:
df

,word,similar_words_collection,translated_word,recall_value,reciprocal_rank
0,aalborg,"[copenhague, odense, göteborg, malmö, trondhei...",aalborg,0,0.0
1,aarhus,"[copenhague, oslo, odense, stockholm, göteborg...",aarhus,0,0.0
2,aba,"[aba, nba, ers, asa, wnba, playoffs, basketbal...",aba,1,1.0
3,abad,"[mohammad, bazar, farah, persan, ispahan, téhé...",abad,0,0.0
4,abandon,"[abandonner, quitter, renoncer, reprendre, ret...",abandon,0,0.0
...,...,...,...,...,...
9864,zoology,"[zoologie, biologie, paléontologie, entomologi...",zoologie,1,1.0
9865,zorro,"[zorro, dracula, bandit, mask, tarzan, serial,...",zorro,1,1.0
9866,zu,"[zu, bei, und, auf, eine, einer, ein, nach, de...",zu,1,1.0
9867,zulu,"[boers, afrikaans, afrika, africain, africaine...",zulu,0,0.0


In [31]:
## TODO: Compute recall@10 and MRR@10 when retrieving 10 nearest neighbors in German and some other language.
df1 = pd.DataFrame(columns = ['word', 'similar_words_collection','translated_word', 'recall_value','reciprocal_rank'])

for row in links:
  if row[1] == 'ar' :
    if row[0] in envec.keys():
      similar_words = mostSimilarTop10(envec.get(row[0]), arvec)
      df1.loc[len(df1.index)] = [row[0],similar_words,row[2], 999,999] 

recall_values = []
reciprocal_rank = []
for translated_word, words in zip(df1['translated_word'], df1['similar_words_collection']):
  if translated_word in words:
    recall_values.append(1)
    reciprocal_rank.append(1 / (words.index(translated_word) + 1))
  else:
    recall_values.append(0)
    reciprocal_rank.append(0)

df1['recall_value'] = recall_values
df1['reciprocal_rank'] = reciprocal_rank
mean_reciprocal_rank_de = sum(df1['reciprocal_rank'])/len(df1)
print(mean_reciprocal_rank_de)

0.2797308513849228


In [32]:
df1

,word,similar_words_collection,translated_word,recall_value,reciprocal_rank
0,a,"[مماثل, صغير, بسيط, وهو, بديل, تجريبي, نموذجية...",a,0,0.000000
1,aachen,"[بريمن, هامبورغ, ستراسبورغ, شتوتغارت, ميونخ, ب...",آخن,0,0.000000
2,aalborg,"[كوبنهاغن, الدنمارك, كيل, ستوكهولم, توركو, ليي...",آلبورغ,0,0.000000
3,aarhus,"[ستوكهولم, كوبنهاغن, الدنمارك, لييج, هامبورغ, ...",آرهوس,0,0.000000
4,aaron,"[دانييل, هارون, جوناثان, جويل, داني, صموئيل, ج...",هارون,1,0.500000
...,...,...,...,...,...
4097,zinc,"[الزنك, النحاس, المنغنيز, الألمنيوم, القصدير, ...",زنك,0,0.000000
4098,zion,"[صهيون, الكرمل, يهودا, صموئيل, المخلص, يهوذا, ...",صهيون,1,1.000000
4099,zionism,"[الصهيونية, الصهيوني, صهيونية, يهودا, موشيه, ا...",صهيونية,1,0.333333
4100,zombie,"[كراش, رعب, الوحش, وحش, الرعب, ديد, مصاص, الجو...",زومبي,0,0.000000


The list of Wikipedia headwords is short enough that a linear scan through the non-English language embeddings takes some time but is feasible. In a production system, you could index the word embeddings using SimHash or some other locality sensitive hashing scheme, as we discussed for duplicate detection, to speed up this process.